# Analyse Exploratoire du fichier myreport_sql_general_full.csv

### Imports (fichier & modules)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math

#import os
#print(os.getcwd())
#print(os.listdir('241016_LogETL'))

In [ ]:
tab = pd.read_csv('../../data/logServer/myreport_sql_general_full.csv', sep=',')

### Le fichier 241016_LogETL.csv : taille, colonnes

In [ ]:
tab.shape

Il y a 171 489 lignes et 28 colonnes dans le dataframe. 

In [ ]:
tab.columns

In [ ]:
tab.info()

In [ ]:
# Calculer le nombre de valeurs uniques pour chaque colonne
nb_valeurs_uniques = tab.nunique()

# Afficher les résultats
print(nb_valeurs_uniques)

In [ ]:
tab.head()

In [ ]:
tab["Date et heure"] = tab["Date et heure"].str.split(" - ").str[0]
tab = tab.iloc[:-5]
tab['Date et heure'] = pd.to_datetime(tab['Date et heure'], format="%d/%m/%Y %H:%M:%S")
start_date = '2024-08-24 23:32:03'
end_date = '2024-10-15 23:31:49'
tab = tab[(tab['Date et heure'] >= start_date) & (tab['Date et heure'] <= end_date)]

In [ ]:
tab.shape

### Réagencement et Suppression des colonnes

In [ ]:
# renommage des colonnes
tab.rename(columns={"Connexions utilisateur(RAW)": "Connexions utilisateur(nb)"}, inplace=True)
tab.rename(columns={"Connexions/s(RAW)": "Connexions/s(nb)"}, inplace=True)
tab.rename(columns={"Déconnexions/s(RAW)": "Déconnexions/s(nb)"}, inplace=True)
tab.rename(columns={"Temps mort(RAW)": "Temps mort sql general(%)"}, inplace=True)
tab.rename(columns={"Couverture(RAW)": "Couverture sql general(%)"}, inplace=True)

In [ ]:
# suppression des colonnes inutiles
tab = tab.drop(columns=["Date et heure(RAW)"])
tab = tab.drop(columns=["Connexions/s"])
tab = tab.drop(columns=["Déconnexions/s"])
tab = tab.drop(columns=["Connexions utilisateur"])
tab = tab.drop(columns=["Temps mort"])
tab = tab.drop(columns=["Couverture"])

In [ ]:
tab.head()

### Analyse temporelle

In [ ]:
# Vérifier que 'date_full' est bien au format datetime
tab["Date et heure"] = pd.to_datetime(tab["Date et heure"])

# Liste des colonnes à exclure
excluded_cols = ["Date et heure"]

# Tracer chaque colonne en fonction du temps
for col in tab.columns:
    if col not in excluded_cols:  # Exclure les colonnes non pertinentes
        plt.figure(figsize=(10, 5))

        plt.plot(tab["Date et heure"], tab[col], marker='o', linestyle='-')

        plt.xlabel("Date et heure")
        plt.ylabel(f"{col}")  # Correction de la syntaxe
        plt.title(f"Évolution de {col} en fonction du temps")
        plt.legend([col])  # Correction de la légende
        plt.xticks(rotation=45)
        plt.grid()

        plt.show()

In [ ]:
# Exclure les colonnes non numériques
numerical_cols = tab.select_dtypes(include=['number']).columns

# Tracer un boxplot pour chaque colonne
for col in numerical_cols:
    if col not in excluded_cols:  # Exclure les colonnes non pertinentes
        plt.figure(figsize=(8, 5))
        sns.boxplot(y=tab[col])
        plt.title(f"Boxplot de {col}")
        plt.grid()
        plt.show()

In [ ]:
# Sélection des colonnes numériques
numerical_cols = tab.select_dtypes(include=['number']).columns

# Filtre pour exclure les colonnes non pertinentes
cols_to_plot = [col for col in numerical_cols if col not in excluded_cols]

rows=(math.ceil(len(cols_to_plot)/3))
# Créer une figure avec 2 lignes et 3 colonnes de sous-graphiques (axes)
fig, axes = plt.subplots(nrows=rows, ncols=3, figsize=(15, rows*5))

# Convertir "axes" en un tableau 1D pour plus de flexibilité
axes = axes.flatten()

# Boucler sur les colonnes et tracer un boxplot dans chaque sous-graphique
for i, col in enumerate(cols_to_plot):
    sns.boxplot(y=tab[col], ax=axes[i])
    axes[i].set_title(f"Boxplot de {col}")
    axes[i].grid()

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout()

# Afficher la figure
plt.show()

### drop des colonnes inutiles

In [ ]:
tab = tab.drop(columns=["Temps mort sql general(%)"])
tab = tab.drop(columns=["Couverture sql general(%)"])

### Enregistrement du nouveau dataframe dans un nouveau csv

In [ ]:
#### ATTENTION au path et nom de fichier
tab.to_csv("../data/logServer/filtered/myreport_sql_general_filtered.csv", index=False, encoding='utf-8')